In [29]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from os.path import join
from tqdm import tqdm
import os
from config import bkg_data_path, signal_data_path

### Listing all the files on the bkg and signal directory (file path)

In [32]:
bkg_files = glob.glob(join(bkg_data_path, "*.*"))
signal_files = glob.glob(join(signal_data_path, "*/*.*"))

print("Signal files:", len(signal_files), "\nBackground files:", len(bkg_files))

Signal files: 5 
Background files: 18
